In [3]:
import torch
import torch.nn as nn 
import torchvision
import torchvision.transforms as transforms

In [2]:
batch_size=64
num_classes=10
learning_rate=0.001
num_epochs=10

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
#  loading the dataset

train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True,
                                           transform=transforms.Compose([
                                               transforms.Resize((32, 32)),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean=(0.1307,), std=(0.3801,))
                                           ]), download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False,
                                          transform=transforms.Compose([
                                              transforms.Resize((32, 32)),
                                              transforms.ToTensor(),
                                              transforms.Normalize(mean=(0.1325,), std=(0.3105,))
                                          ]), download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9.91M/9.91M [00:02<00:00, 3.77MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 119kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.07MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 3.78MB/s]


In [10]:
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0), 
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out


In [11]:
model = LeNet5(num_classes=num_classes).to(device)

# setting the loss function 

cost = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

total_step = len(train_loader)

In [12]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images.to(device)
        labels.to(device)

        # forward pass 
        outputs = model(images)
        loss = cost(outputs, labels)

        # backward pass and optimization 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) %400 == 0:
            print('Epoch [{}/{}], step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], step [400/938], Loss: 0.1759
Epoch [1/10], step [800/938], Loss: 0.1364
Epoch [2/10], step [400/938], Loss: 0.0725
Epoch [2/10], step [800/938], Loss: 0.0142
Epoch [3/10], step [400/938], Loss: 0.0226
Epoch [3/10], step [800/938], Loss: 0.0036
Epoch [4/10], step [400/938], Loss: 0.0931
Epoch [4/10], step [800/938], Loss: 0.0070
Epoch [5/10], step [400/938], Loss: 0.0250
Epoch [5/10], step [800/938], Loss: 0.0248
Epoch [6/10], step [400/938], Loss: 0.0074
Epoch [6/10], step [800/938], Loss: 0.0928
Epoch [7/10], step [400/938], Loss: 0.0149
Epoch [7/10], step [800/938], Loss: 0.0019
Epoch [8/10], step [400/938], Loss: 0.0094
Epoch [8/10], step [800/938], Loss: 0.0042
Epoch [9/10], step [400/938], Loss: 0.0486
Epoch [9/10], step [800/938], Loss: 0.0065
Epoch [10/10], step [400/938], Loss: 0.0225
Epoch [10/10], step [800/938], Loss: 0.0009


In [13]:
# Test the model 

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total+=labels.size(0)
        correct+= (predicted == labels).sum().item()

    print('Accuracy of this network on 10000 test images is: {}%'.format((correct/total)*100))

Accuracy of this network on 10000 test images is: 99.13%
